In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

In [ ]:
custom_transform = transforms.Compose([
    transforms.Resize((224,224)),  # Randomly crop to 224x224
    transforms.RandomHorizontalFlip(),  # Randomly flip horizontally
    transforms.RandomRotation(10),      # Randomly rotate by 10 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Randomly change brightness, contrast, etc.
    transforms.ToTensor(),              # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
])

training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform=custom_transform,
)

train_dataloader = torch.utils.data.DataLoader(training_data, batch_size=64, shuffle=True)

val_data = datasets.Flowers102(
    root="data",
    split="val",
    download=True,
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize the image
]),
)

val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.conv5 = nn.Conv2d(512, 1024, kernel_size=3, padding=1) # Added layer
        self.bn5 = nn.BatchNorm2d(1024)
        self.conv6 = nn.Conv2d(1024, 2048, kernel_size=3, padding=1) # Added layer
        self.bn6 = nn.BatchNorm2d(2048)

        # Fully connected layers
        self.fc1 = nn.Linear(2048 * 3 * 3, 1024)  # Updated to match output of conv layers
        self.bn7 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn8 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 256)
        self.bn9 = nn.BatchNorm1d(256)
        self.fc4 = nn.Linear(256, 128)
        self.bn10 = nn.BatchNorm1d(128)
        self.fc5 = nn.Linear(128, 102)  # Added layer

    def forward(self, x):
        # Apply convolutional layers with ReLU and max pooling
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn5(self.conv5(x)))  # Added layer
        x = F.max_pool2d(x, kernel_size=2, stride=2)  # Pooling layer for added conv layer
        x = F.relu(self.bn6(self.conv6(x)))  # Added layer
        x = F.max_pool2d(x, kernel_size=2, stride=2)  # Pooling layer for added conv layer

        # Flatten the tensor
        x = x.view(-1, 2048 * 3 * 3)  # Updated to match output of conv layers

        # Apply fully connected layers with ReLU
        x = F.relu(self.bn7(self.fc1(x)))
        x = F.relu(self.bn8(self.fc2(x)))
        x = F.relu(self.bn9(self.fc3(x)))
        x = F.relu(self.bn10(self.fc4(x)))
        x = self.fc5(x)  # Added layer

        return F.log_softmax(x, dim=1)

model = NeuralNetwork().to(device)

In [ ]:

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)


In [ ]:
# Training loop
def train_loop(dataloader, model, loss_fn, optimizer, device):
    model.train()
    running_loss = 0.0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(X)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * X.size(0)
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

# Validation loop
def val_loop(dataloader, model, loss_fn, device):
    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    val_loss /= len(dataloader)
    correct /= len(dataloader.dataset)
    return val_loss, correct * 100


In [ ]:
# Training
epochs = 100
start = time.time()
best_val_loss = float('inf')
patience = 10
counter = 0

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer, device)
    val_loss, val_accuracy = val_loop(val_dataloader, model, loss_fn, device)

    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}")

    # Adjust learning rate based on validation loss
    scheduler.step(val_loss)

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping")
            break

end = time.time()
print(f"Training took: {(end - start) / 60:.2f} minutes!")

Epoch 1/100
Train Loss: 4.6686, Val Loss: 4.9614, Val Accuracy: 1.37
Epoch 2/100
Train Loss: 4.4983, Val Loss: 4.3979, Val Accuracy: 3.53
Epoch 3/100
Train Loss: 4.3581, Val Loss: 4.2820, Val Accuracy: 3.33
Epoch 4/100
Train Loss: 4.2399, Val Loss: 4.1407, Val Accuracy: 5.29
Epoch 5/100
Train Loss: 4.1213, Val Loss: 4.1183, Val Accuracy: 6.67
Epoch 6/100
Train Loss: 4.0141, Val Loss: 4.0587, Val Accuracy: 5.39
Epoch 7/100
Train Loss: 3.8915, Val Loss: 3.9373, Val Accuracy: 8.63
Epoch 8/100
Train Loss: 3.8241, Val Loss: 3.8591, Val Accuracy: 7.25
Epoch 9/100
Train Loss: 3.7381, Val Loss: 3.7478, Val Accuracy: 8.53
Epoch 10/100
Train Loss: 3.6458, Val Loss: 3.7851, Val Accuracy: 10.49
Epoch 11/100
Train Loss: 3.5488, Val Loss: 3.7141, Val Accuracy: 10.29
Epoch 12/100
Train Loss: 3.4707, Val Loss: 3.6407, Val Accuracy: 11.86
Epoch 13/100
Train Loss: 3.4252, Val Loss: 3.6597, Val Accuracy: 13.24
Epoch 14/100
Train Loss: 3.2955, Val Loss: 3.6925, Val Accuracy: 12.65
Epoch 15/100
Train Loss: